In [ ]:
# This notebook is used for high-speed ingest from Databricks, and is meant to be submitted as a job.
import argparse

parser = argparse.ArgumentParser(description='Stage data for fast ingest.')
parser.add_argument('access_key_id')
parser.add_argument('secret_access_key')
parser.add_argument('sql_query')
parser.add_argument('stage_output_bucket')
parser.add_argument('stage_output_prefix')

args = parser.parse_args()

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", f"{args.access_key_id}")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", f"{args.secret_access_key}")
sc._jsc.hadoopConfiguration().set(
    "fs.s3a.aws.credentials.provider", 
    "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
)

df = spark.sql(f"""{args.query}""")
df.write \
    .format("csv") \
    .option("compression", "gzip") \
    .option("nullValue", "_SISU_NULL") \
    .option("delimiter", "\x1e") \
    .mode("overwrite") \
    .save(f"s3a://{args.stage_output_bucket}/{args.stage_output_prefix}")

